#### 텍스트 유사도 오픈소스 테스트
paraphrase-MiniLM-L6-v2
- 같은 의미를 갖는 문장 검색
- 모델 크기 22M
all-MiniLM-L6-v2
- 긴 문서에서 유사한 문장 찾기
- 모델 크기 22M
paraphrase-mpnet-base-v2
- paraphrase-MiniLM-L6-v2와 같은 task를 하는 고급 모델
- cpu에서 실행할 수 있음

In [3]:
from sentence_transformers import util, SentenceTransformer
# 🔹 1. 모델 로드
#model = SentenceTransformer("paraphrase-MiniLM-L6-v2", device="cpu")
model = SentenceTransformer("paraphrase-mpnet-base-v2", device="cpu")

# 🔹 2. 질문 및 답변 후보
query = "고양이는 날 수 있다."
documents = [
    "새는 날 수 있는 동물이다.",
    "고양이는 다리가 네 개이다.",
    "대부분 사람들은 고양이가 날 수 없다고 주장한다."
]

# 🔹 3. 임베딩 생성
query_embedding = model.encode(query)
doc_embeddings = model.encode(documents)

# 🔹 4. 유사도 계산 (코사인 유사도)
similarities = util.cos_sim(query_embedding, doc_embeddings)[0]  # (1, N) -> (N,)

# 🔹 5. 유사도가 높은 순서대로 정렬
top_k = 3
top_results = sorted(zip(documents, similarities), key=lambda x: x[1], reverse=True)[:top_k]

# 🔹 6. 결과 출력
for doc, score in top_results:
    print(f"관련 문장: {doc} (유사도: {score:.4f})")

관련 문장: 새는 날 수 있는 동물이다. (유사도: 0.8640)
관련 문장: 고양이는 다리가 네 개이다. (유사도: 0.8375)
관련 문장: 대부분 사람들은 고양이가 날 수 없다고 주장한다. (유사도: 0.7818)


snunlp/KR-SBERT-V40K-klueNLI-augSTS
- Sentence-Bert를 한국어로 fine-tuning한 모델
- 한국어 기준 다른 모델에 비해 성능 좋음
- 모델 크기 420m

In [43]:
import re
from sentence_transformers import SentenceTransformer, util

# 모델 로드 (CPU 실행)
#model = SentenceTransformer("multi-qa-mpnet-base-dot-v1", device="cpu")
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS", device="cpu")

def find_top_k_answers_with_scores(text, query, k=3):
    """정규 표현식을 사용하여 긴 글에서 질문에 대한 상위 k개 답변과 유사도를 함께 찾는 함수"""

    # 문장 분리 (마침표, 물음표, 느낌표 기준으로 분리)
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', text)

    # 문장 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 질문 임베딩 생성
    query_embedding = model.encode(query)

    # 질문과 각 문장 간의 유사도 계산
    similarities = util.cos_sim(query_embedding, sentence_embeddings)[0]

    # 문장과 유사도를 함께 저장
    sentence_scores = list(zip(sentences, similarities))

    # 유사도 기준으로 내림차순 정렬
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # 상위 k개 문장과 유사도 반환
    top_k_sentences_with_scores = sentence_scores[:k]

    return top_k_sentences_with_scores

# 예시 긴 글
long_text = """
장제원 전 국민의힘 의원이 숨진 채 발견됐다.

1일 경찰 등에 따르면 장 전 의원은 전날 오후 11시 40분경 서울 강동구의 한 오피스텔에서 숨진 채 발견됐다.

현장에는 장 전 의원이 작성한 것으로 보이는 유서가 발견된 것으로 전해졌다. 유서에는 “가족들에게 미안하다” “사랑한다” 등 가족을 향한 내용이 담겼던 것으로 알려졌다.

현재까지 타살 정황은 없는 것으로 알려졌으며, 경찰은 정확한 사망 경위를 조사 중이다.

장 전 의원은 2015년 11월 부산의 한 대학 부총장으로 재직할 당시 비서를 성폭행한 혐의로 고소됐다.

고소인은 경찰 조사에서 “장 전 의원의 총선 출마를 앞두고 선거 포스터를 촬영한 뒤 뒤풀이 자리에서 술을 마셨다. 이후 자신에게 성폭력을 저질렀다”고 진술한 것으로 알려졌다. 또 사건 피해 직후 장 전 의원으로부터 ‘그렇게 가면 내 마음은 어떡해’라는 내용의 문자메시지를 받았으며, 이를 보관하고 있다가 경찰에 제출한 것으로 알려졌다.

경찰은 지난달 28일 준강간치상 혐의로 고소당한 장 전 의원을 소환했다. 장 전 의원은 경찰 조사에서 혐의를 전면 부인한 것으로 알려졌다.

장 전 의원 측은 2일부터 부산해운대백병원에 빈소를 마련하고 조문을 받기로 했다.
"""

# 질문
query = "장제원 의원은 타살당했나요?"

# 상위 2개 답변과 유사도 찾기
top_answers_with_scores = find_top_k_answers_with_scores(long_text, query, k=3)

print("질문:", query)
print("상위 답변:")
for sentence, score in top_answers_with_scores:
    print("- 문장:", sentence)
    print("  유사도:", score)

질문: 장제원 의원은 타살당했나요?
상위 답변:
- 문장: 
장제원 전 국민의힘 의원이 숨진 채 발견됐다.
  유사도: tensor(0.5864)
- 문장: 
현재까지 타살 정황은 없는 것으로 알려졌으며, 경찰은 정확한 사망 경위를 조사 중이다.
  유사도: tensor(0.5236)
- 문장: 
현장에는 장 전 의원이 작성한 것으로 보이는 유서가 발견된 것으로 전해졌다.
  유사도: tensor(0.3695)


단순하게 단어 겹치는 정도로 비교한 유사도

In [20]:

def jaccard_similarity(sentence1, sentence2):
    
    set1 = set(sentence1.split())
    set2 = set(sentence2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union else 0

query = "고양이는 날 수 있다."
documents = [
    "새는 날 수 있는 동물이다.",
    "고양이는 다리가 네 개이다.",
    "대부분 사람들은 고양이가 날 수 없다고 주장한다."
]

similarities = []
for doc in documents:
    similarity = jaccard_similarity(query, doc)
    similarities.append((doc, similarity))

# 유사도 기준으로 내림차순 정렬
similarities.sort(key=lambda x: x[1], reverse=True)

print("자카드 유사도 순위:")
for doc, similarity in similarities:
    print(f"- 문장: {doc}")
    print(f"  유사도: {similarity}")

자카드 유사도 순위:
- 문장: 새는 날 수 있는 동물이다.
  유사도: 0.2857142857142857
- 문장: 대부분 사람들은 고양이가 날 수 없다고 주장한다.
  유사도: 0.2222222222222222
- 문장: 고양이는 다리가 네 개이다.
  유사도: 0.14285714285714285


snunlp/KR-SBERT-V40K-klueNLI-augSTS 모델을 사용하여 기사에서 질문과 관련된 문장 추출 테스트

In [42]:
import re
from sentence_transformers import SentenceTransformer, util

# 모델 로드 (CPU 실행)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS", device="cpu")

def find_top_k_answers_regex(text, query, k=3):
    """정규 표현식을 사용하여 긴 글에서 질문에 대한 상위 k개 답변을 찾는 함수"""

    # 문장 분리 (마침표, 물음표, 느낌표 기준으로 분리)
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', text)

    # 문장 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 질문 임베딩 생성
    query_embedding = model.encode(query)

    # 질문과 각 문장 간의 유사도 계산
    similarities = util.cos_sim(query_embedding, sentence_embeddings)[0]

    # 유사도와 문장 인덱스를 함께 저장
    sentence_scores = list(zip(sentences, similarities))

    # 유사도 기준으로 내림차순 정렬
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # 상위 k개 문장 반환
    top_k_sentences_with_scores = sentence_scores[:k]

    return top_k_sentences_with_scores

# 예시 긴 글
long_text = """
배우 김수현이 지난달 16일 세상을 떠난 고(故) 김새론 배우가 성인이 된 후 1년 간 교제를 했다고 밝혔다. 논란이 불거진 지 약 3주 만에 처음으로 공개 석상에 모습을 드러낸 그는, 미성년자 교제 의혹을 부인하며 억울함을 호소했다.

김수현의 소속사 골드메달리스트는 31일 오후 4시 30분 서울 마포구 상암동 스탠포드호텔에서 긴급 기자회견을 열었다.

김수현은 “먼저 죄송하다. 저 한 사람 때문에 너무 많은 분들이 고통받고 있는 것 같다. 그리고 고인도 편히 잠들지 못하고 있는 것 같아서 안타까운 마음뿐이다”라며 입을 열었다.

이어 김수현은 “처음부터 이 자리에서 모든 걸 다 이야기하면 어땠을까 하는 생각을 했다. 그랬으면 저를 사랑해주신 팬분들, 이 기자회견까지 말할 수 없이 애써주신 회사 식구분들, 다 이토록 괴롭지는 않지 않았을까”라며 “저와 고인의 사생활이 폭로될 때마다 내일은 그냥 다 이야기하자, 직접 말하고 이 지옥 같은 상황을 끝내자는 생각을 계속 했었다”라고 이야기했다.

그는 김새론과의 교제 사실은 인정하면서도, 미성년자 시절부터 만남을 가졌다는 의혹에 대해서는 강하게 부인했다. 그는 “저와 고인은 5년 전, 드라마 눈물의 여왕이 방영되기 4년 전에 약 1년간 교제했다”며 “고인이 미성년자이던 시절 교제를 하지 않았다”고 강력히 주장했다.

또한, 고인의 사망 원인이 소속사의 압박 때문이라는 주장에도 반박했다. 그는 “고인이 저의 외면으로 인해, 저희 소속사가 고인의 채무를 압박했기 때문에 비극적인 선택을 했다는 것 또한 사실이 아니다”며 “평범한 연인이었고, 서로 좋은 감정을 갖고 만났으나 시간이 지나 헤어졌을 뿐”이라고 말했다.

김수현은 “저희 소속사가 고인과의 채무 관계에 대해 2차 내용증명을 보내 고인에게 채무에 대해 압박된 것처럼 말했다. 하지만 1년 전 제 소속사 대표와의 통화에서는 전혀 다른 말을 하고 있었다”라며 고인의 유족과 소속사 대표와의 통화 녹음본을 공개했다.

이어 김수현은 “고인의 마지막 소속사 대표가 1년 전 통화와 완전히 다른 거짓말을 하고 있다. 제가 잘못한 일은 얼마든지 인정하겠다. 책임져야 할 일이 있다면 책임지는게 당연하다고 생각한다”라고 말했다.

김수현은 고인과의 관계가 공개된 이후, 협박과 거짓된 증거들이 계속 등장했다고 주장했다. 그는 “날마다 새로운 폭로가 예고되고, 거짓을 사실처럼 인정하라는 강요를 받았지만 하지 않은 일을 했다고 말할 수는 없다”고 단호한 입장을 밝혔다.

특히, 고인의 유족 측이 제시한 증거에 대해 의문을 제기하며 이를 검증할 것을 요청했다. 김수현은 “유족이 공개한 2016년과 2018년 카카오톡 대화 속 인물은 서로 다른 사람이라는 분석 결과를 검증 기관을 통해 확인했다”며 “2016년에 촬영됐다고 밝힌 사진은 2019년에 촬영된 사진이며, 4년간 몸 담았던 소속사 이름과 계약기간 역시 틀렸다”고 밝혔다.

그러면서 “유족 측이 가진 증거가 정말 진실이라면, 수사기관에 모든 자료를 제출하고 법적인 절차를 통해 검증받을 것을 요청한다”라고 요구했다.

끝으로 그는 “제가 한 일은 어떤 비난도 감수하겠다”면서도 “하지만, 하지 않은 것은 하지 않은 것이다. 저를 믿어주시는 모든 분들을 위해서 그것만큼은 밝히고 싶다”며 결백을 주장했다.

이날 기자회견에는 김수현과 소속사 골드메달리스트의 법률대리인인 김종복 변호사도 동행해 공식 입장을 밝혔다. 김 변호사는 “김수현 배우가 직접 입장을 표명한 만큼, 사실관계를 명확히 밝히고자 관련자들에 대한 형사 고소 및 민사 소송을 제기하기로 했다”고 전했다.

이어 “오늘 유족과 이모라고 주장하는 설명불상자, 그리고 가로세로연구소(가세연) 운영자를 상대로 정보통신망법 위반(명예훼손) 혐의로 고소장을 제출할 예정이며, 120억 원 상당의 손해배상 청구 소송도 서울중앙지방법원에 제출할 계획”이라고 밝혔다.

또한 “현재 이 사안은 수사 대상이기 때문에 추가적인 질의응답을 받지 않는다”며 “이 점을 양해해달라”고 덧붙였다.

기자회견을 마친 후 김수현이 회견장을 떠나려 하자, 한 취재진이 “김새론을 언제 어디서 처음 만났는지 말해 달라. 이 정도는 말해 줄 수 있지 않느냐”고 질문했다. 그러나 김수현은 별다른 답변을 하지 않은 채 자리를 떠났다.

한편, 김새론은 지난달 16일 오후 4시 54분경 서울 성동구 소재 자택에서 숨진 채 발견됐다. 유족 측은 지난 10일 유튜브 채널 ‘가로세로 연구소’(가세연)를 통해 김새론이 만 15세였던 2015년 11월 19일부터 2021년 7월 7일까지 약 6년간 김수현과 교제했다고 주장했다.

이후 김수현 측은 김새론이 성인이던 2019년 여름부터 2020년 가을까지 교제했다고 반박에 나섰다. 이에 김새론 유족 측은 27일 법률대리인 부지석 법무법인 부유 대표변호사와 기자회견을 열어 김새론이 만 16세 시절 김수현과 나눈 카카오톡 메시지와 편지 내용을 공개했다.
"""

# 질문
query = "그래서 김수현이랑 김새론이랑 미성년자일 때 사귄거지?"

# 상위 3개 답변과 유사도 찾기
top_answers_with_scores = find_top_k_answers_regex(long_text, query, k=3)

# 결과 출력
for sentence, score in top_answers_with_scores:
    print(f"관련 문장: {sentence} (유사도: {score:.4f})")

관련 문장: 
배우 김수현이 지난달 16일 세상을 떠난 고(故) 김새론 배우가 성인이 된 후 1년 간 교제를 했다고 밝혔다. (유사도: 0.6663)
관련 문장: 
이후 김수현 측은 김새론이 성인이던 2019년 여름부터 2020년 가을까지 교제했다고 반박에 나섰다. (유사도: 0.6038)
관련 문장: 
그는 김새론과의 교제 사실은 인정하면서도, 미성년자 시절부터 만남을 가졌다는 의혹에 대해서는 강하게 부인했다. (유사도: 0.5509)


NLI task 테스트
- 기준 문장 하나와 비교할 문장 여러개가 있을 때 각 비교 문장이 기준 문장에 대해서 찬(entailment), 반(contradiction), 중립(neutral)으로 판단
- 아래 모델들에 대해서 테스트를 하였으나 실행할 때마다 다른 결과가 나오고 결과가 정확하지 않음
- 영어 모델로 실행한 뒤 번역하는 방식 테스트 가능

In [152]:
from transformers import pipeline

# NLI 모델 로드
#nli_model = pipeline("zero-shot-classification", model="snunlp/KR-SBERT-V40K-klueNLI-augSTS")  
#nli_model = pipeline("zero-shot-classification", model="beomi/kcbert-base")
#nli_model = pipeline("zero-shot-classification", model="monologg/koelectra-base-v3-discriminator")
#nli_model = pipeline("zero-shot-classification", model="klue/bert-base")
#nli_model = pipeline("zero-shot-classification", model="klue/roberta-base")
#nli_model = pipeline("zero-shot-classification", model="jhgan/ko-sbert-nli")
nli_model = pipeline("zero-shot-classification", model="klue/roberta-large")

# 기본 문장(참/거짓을 판별하고 싶은 문장)
claim = "커피는 잠을 깨운다."

# 증거 문장(참고할 문장들)
evidence_sentences = [
    "카페인은 중추 신경계를 자극하여 졸음을 줄인다.",
    "커피를 마시면 오히려 피곤해질 수도 있다.",
    "잠을 깨우는 효과는 개인차가 있다."
]

# NLI 모델에서 사용할 라벨 (참: "entailment", 거짓: "contradiction", 중립: "neutral")
labels = ["entailment", "contradiction", "neutral"]

# 각 증거 문장과 비교하여 참/거짓 판별
for evidence in evidence_sentences:
    result = nli_model(evidence, candidate_labels=labels)
    print(f"증거 문장: {evidence} → 결과: {result['labels'][0]} (신뢰도: {result['scores'][0]:.4f})")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


증거 문장: 카페인은 중추 신경계를 자극하여 졸음을 줄인다. → 결과: contradiction (신뢰도: 0.3346)
증거 문장: 커피를 마시면 오히려 피곤해질 수도 있다. → 결과: contradiction (신뢰도: 0.3347)
증거 문장: 잠을 깨우는 효과는 개인차가 있다. → 결과: contradiction (신뢰도: 0.3377)


In [6]:
from transformers import pipeline

# zero-shot classification 파이프라인 로드
nli_model = pipeline("zero-shot-classification", model="beomi/kcbert-base")

# 기본 문장(참/거짓을 판별하고 싶은 문장)
claim = "장제원 의원은 타살당함"

# 증거 문장(참고할 문장들)
evidence_sentences = [
    "현재까지 타살 정황은 없는 것으로 알려졌으며, 경찰은 정확한 사망 경위를 조사 중이다.",
    "장제원 전 국민의힘 의원이 숨진 채 발견됐다.",
    "현장에는 장 전 의원이 작성한 것으로 보이는 유서가 발견된 것으로 전해졌다."
]

# zero-shot classification을 위한 후보 레이블
labels = ["ENTAILMENT", "CONTRADICTION", "NEUTRAL"]

# 각 증거 문장과 비교하여 참/거짓 판별
for evidence in evidence_sentences:
    result = nli_model(evidence, candidate_labels=labels, hypothesis=claim)
    
    print(f"증거 문장: {evidence}")
    # 결과를 각 레이블과 그에 대응하는 신뢰도와 함께 출력
    for label, score in zip(result['labels'], result['scores']):
        print(f"레벨: {label} (신뢰도: {score:.4f})")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


증거 문장: 현재까지 타살 정황은 없는 것으로 알려졌으며, 경찰은 정확한 사망 경위를 조사 중이다.
레벨: NEUTRAL (신뢰도: 0.3376)
레벨: ENTAILMENT (신뢰도: 0.3314)
레벨: CONTRADICTION (신뢰도: 0.3311)
증거 문장: 장제원 전 국민의힘 의원이 숨진 채 발견됐다.
레벨: CONTRADICTION (신뢰도: 0.3583)
레벨: ENTAILMENT (신뢰도: 0.3578)
레벨: NEUTRAL (신뢰도: 0.2839)
증거 문장: 현장에는 장 전 의원이 작성한 것으로 보이는 유서가 발견된 것으로 전해졌다.
레벨: ENTAILMENT (신뢰도: 0.3407)
레벨: CONTRADICTION (신뢰도: 0.3344)
레벨: NEUTRAL (신뢰도: 0.3249)


영어 모델로 테스트 NLI task 테스트<br>
영어 문장은 구글 번역기 사용 

In [7]:
from transformers import pipeline

# NLI 파이프라인 생성
nli_pipeline = pipeline("text-classification", model="roberta-large-mnli")

# claim = "Coffee keeps you awake."
# evidence_sentences = [
#     "Caffeine stimulates the central nervous system and reduces drowsiness.",
#     "Drinking coffee can actually make you tired.",
#     "The effect of waking up varies from person to person."
#]

claim = "Rep. Jang Je-won was murdered"
evidence_sentences = [
    "There are currently no signs of murder, and police are investigating the exact circumstances of death.",
    "Former People Power Party lawmaker Jang Je-won was found dead.",
    "It was reported that a suicide note believed to have been written by former lawmaker Jang was discovered at the scene."
]

# 결과 출력
for evidence in evidence_sentences:
    # NLI 입력은 두 문장을 함께 넣어야 함 → "premise", "hypothesis"
    input_text = f"{evidence} [SEP] {claim}"
    result = nli_pipeline(input_text)[0]

    print(f"Evidence: {evidence}")
    print(f"→ Prediction: {result['label']} (Confidence: {result['score']:.4f})\n")


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


Evidence: There are currently no signs of murder, and police are investigating the exact circumstances of death.
→ Prediction: CONTRADICTION (Confidence: 0.9164)

Evidence: Former People Power Party lawmaker Jang Je-won was found dead.
→ Prediction: NEUTRAL (Confidence: 0.8665)

Evidence: It was reported that a suicide note believed to have been written by former lawmaker Jang was discovered at the scene.
→ Prediction: NEUTRAL (Confidence: 0.7546)

